## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Janakpur,NP,26.7183,85.9065,66.18,40,0,6.55,clear sky
1,1,Lorengau,PG,-2.0226,147.2712,81.54,74,95,1.90,overcast clouds
2,2,Kyaukse,MM,21.6000,96.1333,78.66,37,10,4.23,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,49.66,62,0,14.97,clear sky
4,4,Bluff,NZ,-46.6000,168.3333,62.80,69,100,10.98,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Lorengau,PG,-2.0226,147.2712,81.54,74,95,1.90,overcast clouds
2,2,Kyaukse,MM,21.6000,96.1333,78.66,37,10,4.23,clear sky
5,5,Busselton,AU,-33.6500,115.3333,70.61,52,66,9.89,broken clouds
11,11,Half Moon Bay,US,37.4636,-122.4286,70.02,25,0,11.50,clear sky
12,12,Riyadh,SA,24.6877,46.7219,86.14,14,38,17.34,scattered clouds
13,13,Ahipara,NZ,-35.1667,173.1667,71.73,66,42,9.64,scattered clouds
14,14,Avarua,CK,-21.2078,-159.7750,82.45,83,75,9.22,broken clouds
20,20,Mexico,PH,15.0646,120.7198,75.00,80,100,4.65,overcast clouds
23,23,Vaini,TO,-21.2000,-175.2000,78.96,89,75,6.91,thunderstorm
24,24,Rikitea,PF,-23.1203,-134.9692,78.64,70,11,7.43,few clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 272 entries, 1 to 674
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              272 non-null    int64  
 1   City                 272 non-null    object 
 2   Country              268 non-null    object 
 3   Lat                  272 non-null    float64
 4   Lng                  272 non-null    float64
 5   Max Temp             272 non-null    float64
 6   Humidity             272 non-null    int64  
 7   Cloudiness           272 non-null    int64  
 8   Wind Speed           272 non-null    float64
 9   Current Description  272 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 23.4+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 1 to 671
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              268 non-null    int64  
 1   City                 268 non-null    object 
 2   Country              268 non-null    object 
 3   Lat                  268 non-null    float64
 4   Lng                  268 non-null    float64
 5   Max Temp             268 non-null    float64
 6   Humidity             268 non-null    int64  
 7   Cloudiness           268 non-null    int64  
 8   Wind Speed           268 non-null    float64
 9   Current Description  268 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 23.0+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lorengau,PG,81.54,overcast clouds,-2.0226,147.2712,
2,Kyaukse,MM,78.66,clear sky,21.6000,96.1333,
5,Busselton,AU,70.61,broken clouds,-33.6500,115.3333,
11,Half Moon Bay,US,70.02,clear sky,37.4636,-122.4286,
12,Riyadh,SA,86.14,scattered clouds,24.6877,46.7219,
13,Ahipara,NZ,71.73,scattered clouds,-35.1667,173.1667,
14,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
20,Mexico,PH,75.00,overcast clouds,15.0646,120.7198,
23,Vaini,TO,78.96,thunderstorm,-21.2000,-175.2000,
24,Rikitea,PF,78.64,few clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 1 to 671
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 268 non-null    object 
 1   Country              268 non-null    object 
 2   Max Temp             268 non-null    float64
 3   Current Description  268 non-null    object 
 4   Lat                  268 non-null    float64
 5   Lng                  268 non-null    float64
 6   Hotel Name           268 non-null    object 
dtypes: float64(3), object(4)
memory usage: 24.9+ KB


In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lorengau,PG,81.54,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
2,Kyaukse,MM,78.66,clear sky,21.6000,96.1333,Shwe Thit Sar Guesthouse
5,Busselton,AU,70.61,broken clouds,-33.6500,115.3333,Observatory Guest House
11,Half Moon Bay,US,70.02,clear sky,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
12,Riyadh,SA,86.14,scattered clouds,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
...,...,...,...,...,...,...,...
662,Conceicao Do Araguaia,BR,76.60,light rain,-8.2578,-49.2647,Tarumã Tropical Hotel
665,Upington,ZA,81.23,broken clouds,-28.4478,21.2561,River Place Manor
667,Chowchilla,US,71.02,clear sky,37.1230,-120.2602,Days Inn by Wyndham Chowchilla Gateway to Yose...
668,Ewa Beach,US,84.15,clear sky,21.3156,-158.0072,Base Lodging


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [17]:
locations

,Lat,Lng
1,-2.0226,147.2712
2,21.6000,96.1333
5,-33.6500,115.3333
11,37.4636,-122.4286
12,24.6877,46.7219
...,...,...
662,-8.2578,-49.2647
665,-28.4478,21.2561
667,37.1230,-120.2602
668,21.3156,-158.0072


In [18]:
# 11a. Add a marker layer for each city to the map. 

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto',
    'zoom_level':1.5
}
fig = gmaps.figure(layout=figure_layout)

# Access maps with unique API key
gmaps.configure(api_key=g_key)
locations = hotel_df[["Lat", "Lng"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)


# 11b. Display the figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…